In [ ]:
from hope.models.segmodel import SegModel
import torch
from hope.utils.embed import create_embs_from_names
import cv2
from hope.utils.transform_utils import resize_by_scaled_short_side
import json
from hope.utils.transform_utils import single_scale_single_crop_cuda


In [ ]:
model = SegModel(criterions=None,
                     num_classes=512,
                     load_imagenet_model=False, imagenet_ckpt_fpath='')


model = model.eval()

model = torch.nn.DataParallel(model)


In [ ]:
checkpoint = torch.load('/home/penguin/SSIW/data/model/segformer_7data.pth', map_location='cpu')['state_dict']
ckpt_filter = {k: v for k, v in checkpoint.items() if 'criterion.0.criterion.weight' not in k}
maybe_errors = model.load_state_dict(ckpt_filter, strict=False)

In [ ]:
model = model.to('cuda')

In [ ]:
with open('/home/penguin/SSIW/Test_Minist/cmp_dataset_desc.json', 'r', encoding='utf-8') as f:
    new_definitions = json.load(f)

In [ ]:
user_label = ['background','facade','window','door', 'cornice', 'sill', 'balcony', 'blind','deco', 'molding', 'pillar', 'shop', 'unlabel']

In [ ]:
gt_embs_list = create_embs_from_names(user_label, new_definitions).float()

In [ ]:
rgb = cv2.imread('/home/penguin/SSIW/data/base/cmp_b0001.jpg', -1)[:, :, ::-1]

In [ ]:
image_resized = resize_by_scaled_short_side(rgb, 720, 1)
h, w, _ = rgb.shape

In [ ]:
out_logit, emb = single_scale_single_crop_cuda(model, image_resized, h, w, gt_embs_list=gt_embs_list, args=None)

In [ ]:
import numpy as np

In [ ]:
id_to_label = {i: v for i, v in enumerate(user_label)}
id_to_label[255] = 'unlabel'

In [ ]:
prediction.shape

In [ ]:
prediction

In [ ]:
prediction = out_logit.argmax(axis=-1).squeeze()   # (h, w)
probs = out_logit.max(axis=-1).squeeze()  # (h, w)
high_prob_mask = probs > 0.5

mask = high_prob_mask
prediction[~mask] = 255

labels = np.unique(prediction)
label_names = {i: id_to_label[i] for i in labels}
# label_names = [id_to_label[i] for i in labels]
print()
print(f'label_names for img', label_names)

In [ ]:
id_to_label

In [ ]:
prediction.shape

In [ ]:
import torch.nn as nn
import torch

In [ ]:
loss = nn.CrossEntropyLoss()

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

In [ ]:
target

In [ ]:
input

In [ ]:
import xml.etree.ElementTree as ET
from dataclasses import dataclass


@dataclass
class Point:
    x1: float
    x2: float
    y1: float
    y2: float


@dataclass
class ObjectMetadata:
    points: Point
    label: int
    label_name: str
    flag: int


def parse_cmp_string_xml(chunk: list):
    et = ET.fromstring(' '.join(chunk))

    points, label, labelname, flag = list(et)
    points = [float(point.text.strip()) for point in points]
    label = int(label.text.strip())
    labelname = labelname.text.strip()
    flag = int(flag.text.strip())

    return ObjectMetadata(Point(*points), label, labelname, flag)


def parse_cmp_xml(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.readlines()

    metadatas = []
    chunk = []

    for element in data:
        if element.startswith('</object>'):
            chunk.append(element)
            metadata = parse_cmp_string_xml(chunk)
            chunk.clear()
            metadatas.append(metadata)
        else:
            chunk.append(element)

    return metadatas

In [ ]:
metadatas = parse_cmp_xml('/home/penguin/SSIW/data/base/cmp_b0002.xml')

In [ ]:
obj = metadatas[1]

In [ ]:
obj

In [ ]:
from PIL import Image, ImageDraw

# Load the image

In [ ]:
img = Image.open("/home/penguin/SSIW/data/base/cmp_b0002.jpg")

In [ ]:
width, height = 398, 1024

In [ ]:
obj.points.x1 *= width
obj.points.x2 *= width
obj.points.y1 *= height
obj.points.y2 *= height


In [ ]:
obj

In [ ]:
from PIL import Image, ImageDraw

# Define the rectangle coordinates
x0, y0 = obj.points.x1, obj.points.y1
x1, y1 = obj.points.x2, obj.points.y2

# Draw the rectangle
draw = ImageDraw.Draw(img)
draw.rectangle((x0, y0, x1, y1), outline="red")

# Show the image
img.show()

In [ ]:
import cv2 
   
# path 
path = r'/home/penguin/SSIW/data/base/cmp_b0001.jpg'
   
# Reading an image in default mode
image = cv2.imread(path)

In [ ]:
start_point = (int(obj.points.x1), int(obj.points.x2))
  
# Ending coordinate, here (220, 220)
# represents the bottom right corner of rectangle
end_point = (int(obj.points.y1), int(obj.points.y2))
  
# Blue color in BGR
color = (255, 0, 0)
  
# Line thickness of 2 px
thickness = 2
  
# Using cv2.rectangle() method
# Draw a rectangle with blue line borders of thickness of 2 px
image = cv2.rectangle(image, start_point, end_point, color, thickness)

cv2.imshow('test', image)

In [ ]:
from hope.dataset.dataset import CMPFacadeDataset